In [ ]:
!pip install gradio ultralytics pillow

import gradio as gr
from PIL import Image, ImageDraw
import os

# ---------------------------------------------------
#          LOAD YOUR CUSTOM MODEL (best.pt)
# ---------------------------------------------------
MODEL_PATH = "/content/best (1).pt"  # <- استخدمي اسم الملف الصحيح هنا
CONF_THRESHOLD = 0.25

model = None
ULTRA = False

try:
    from ultralytics import YOLO
    ULTRA = True

    if os.path.exists(MODEL_PATH):
        model = YOLO(MODEL_PATH)
        print("Loaded your custom model:", MODEL_PATH)
        print("CLASSES:", model.names)
    else:
        raise FileNotFoundError(f"{MODEL_PATH} not found")

except Exception as e:
    ULTRA = False
    print("Error loading model:", e)
    print("Running in DEMO mode.")


# ---------------------------------------------------
#                 DETECTION FUNCTION
# ---------------------------------------------------
def detect_cavities(image):

    if image is None:
        return None, "Please upload an image first."

    # Ensure image is RGB
    image = image.convert("RGB")
    annotated = image.copy()
    draw = ImageDraw.Draw(annotated)

    # Lists for storing detection details
    cavity_details = []
    normal_details = []

    cavity_count = 0
    normal_count = 0

    if ULTRA and model:
        try:
            results = model.predict(image, conf=CONF_THRESHOLD, verbose=False)
            boxes = results[0].boxes

            for box in boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                name = results[0].names[cls].lower()

                x1, y1, x2, y2 = box.xyxy[0].tolist()

                # ---------------- CAVITY ----------------
                if name in ["cavity", "caries"]:
                    cavity_count += 1
                    cavity_details.append(f"Cavity detected ({conf:.2%})")

                    draw.rectangle([x1, y1, x2, y2], outline="red", width=4)
                    draw.text((x1, y1 - 18), f"Cavity {conf:.2f}", fill="red")

                # ---------------- NORMAL ----------------
                else:
                    normal_count += 1
                    normal_details.append(f"Normal detected ({conf:.2%})")

                    draw.rectangle([x1, y1, x2, y2], outline="green", width=4)
                    draw.text((x1, y1 - 18), f"Normal {conf:.2f}", fill="green")

            # ---------------------------------------------------
            #                   REPORT SECTION
            # ---------------------------------------------------
            report = "DETECTION REPORT\n" + "="*40 + "\n"
            report += f"Normal Teeth: {normal_count}\n"
            report += f"Cavity Teeth: {cavity_count}\n\n"

            # Cavity details
            if cavity_details:
                report += "Cavity Details:\n"
                for d in cavity_details:
                    report += f"- {d}\n"
            else:
                report += "No cavities detected.\n"

            # Normal details
            if normal_details:
                report += "\nNormal Details:\n"
                for d in normal_details:
                    report += f"- {d}\n"

            return annotated, report

        except Exception as e:
            return None, f"ERROR during detection: {e}"

    # ---------------- DEMO MODE ----------------
    report = "DETECTION REPORT (DEMO MODE)\n"
    report += "="*40 + "\nNormal Teeth: 2\nCavity Teeth: 1\n"
    report += "(Demo mode – Model not loaded correctly.)"

    return annotated, report



# ---------------------------------------------------
#                   PROFESSIONAL UI DESIGN
# ---------------------------------------------------
css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700&display=swap');
* { font-family: 'Inter', sans-serif !important; }

body, .gradio-container {
    background-color: #F2F6FB !important;
}

.card {
    background: white;
    padding: 40px;
    border-radius: 18px;
    box-shadow: 0 6px 20px rgba(0,0,0,0.08);
    max-width: 800px;
    margin: auto;
}

.title {
    font-size: 36px;
    font-weight: 700;
    color: #1E4E79;
    text-align: center;
}

.subtitle {
    font-size: 17px;
    color: #5A6C7D;
    text-align: center;
    margin-bottom: 25px;
}

#detect-btn {
    background-color: #1E4E79 !important;
    color: white !important;
    padding: 12px 20px;
    border-radius: 10px !important;
    font-weight: 600 !important;
    font-size: 17px !important;
}
"""

# ---------------------------------------------------
#                        UI
# ---------------------------------------------------
with gr.Blocks(css=css) as interface:
    with gr.Column(elem_classes="card"):
        gr.HTML("<div class='title'>🦷 Dental Cavity Detection System</div>")
        gr.HTML("<div class='subtitle'>Upload a dental RGB image to detect cavities.</div>")

        uploaded_image = gr.Image(label="Upload Dental Image", type="pil")
        detect_button = gr.Button("🔍 Start Detection", elem_id="detect-btn")

        result_image = gr.Image(label="Detection Result")
        report_text = gr.Textbox(label="Detection Report", lines=15)

        detect_button.click(
            fn=detect_cavities,
            inputs=uploaded_image,
            outputs=[result_image, report_text]
        )

interface.launch(share=True)


Loaded your custom model: /content/best (1).pt
CLASSES: {0: 'cavity', 1: 'normal'}


/tmp/ipython-input-4057611794.py:163: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as interface:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30410647236519b4c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Re-attempt to load the model with the new path
from ultralytics import YOLO

model = None
ULTRA = False

try:
    ULTRA = True
    if os.path.exists(MODEL_PATH):
        model = YOLO(MODEL_PATH)
        print("Loaded your custom model:", MODEL_PATH)
    else:
        raise FileNotFoundError(f"{MODEL_PATH} not found")

except Exception as e:
    ULTRA = False
    print("Error loading model:", e)
    print("Running in DEMO mode.")


Loaded your custom model: /content/best (1).pt
